In [1]:
import pandas as pd
import geopandas as gpd

import folium
import branca

import topojson


In [2]:
#read shapes

krs_path = "VG250_KRS.shp"

krs= gpd.read_file(krs_path)

krs = krs.to_crs({'init': 'epsg:4326'})
krs = krs[krs.GF==4].copy()

len(krs)

401

In [3]:
#simplify shapes

#krs.geometry= krs.buffer(0) #repair geometry

krs_tj = topojson.Topology(krs, prequantize=False, topology=True)
krs_tj_sim = krs_tj.toposimplify(0.02)

krs_tj_sim = krs_tj_sim.to_gdf()
krs_tj_sim.crs={'init': 'epsg:4326'}


In [4]:
##### DATA #####

In [5]:
path_data = "BKA-LKS-F-03-T01-Kreise_csv.csv"
data= pd.read_csv(path_data, skiprows=1,sep=";", encoding='latin-1')
data.head()

,Schluessel,Straftat,Gemeindeschluessel,Stadt-/Landkreis,Kreisart,erfasste Faelle,HZ nach Zensus,Versuche - Anzahl,Versuche - Anteil in %,mit Schusswaffe gedroht,mit Schusswaffe geschossen,aufgeklaerte Faelle,Aufklaerungsquote,Tatverdaechtige insgesamt,Tatverdaechtige - maennlich,Tatverdaechtige - weiblich,Nichtdeutsche Tatverdaechtige - Anzahl,Nichtdeutsche Tatverdaechtige - Anteil in %
0,------,Straftaten insgesamt,1001,Flensburg,KfS,"8,156","9,214",633,7.8,11,5,"4,878",59.8,"3,474","2,635",839,"1,178",33.9
1,------,Straftaten insgesamt,1002,Kiel,KfS,"24,621","9,930","1,787",7.3,13,8,"11,941",48.5,"7,877","6,065","1,812","2,251",28.6
2,------,Straftaten insgesamt,1003,Lübeck,KfS,"22,685","10,487","1,682",7.4,21,21,"12,727",56.1,"7,858","5,940","1,918","2,068",26.3
3,------,Straftaten insgesamt,1004,Neumünster,KfS,"10,688","13,472",897,8.4,12,4,"6,660",62.3,"5,061","3,738","1,323","2,465",48.7
4,------,Straftaten insgesamt,1051,Dithmarschen,K,"7,224","5,413",440,6.1,4,13,"4,163",57.6,"3,169","2,392",777,577,18.2


In [6]:
allStraf= list(data.Straftat.unique())
allStraf

['Straftaten insgesamt',
 'Vergewaltigung, sexuelle Nötigung und sexueller Übergriff im besonders schweren Fall einschl. mit Todesfolge §§ 177, 178 StGB',
 'Raub, räuberische Erpressung und räuberischer Angriff auf Kraftfahrer §§ 249-252, 255, 316a StGB',
 'Raub, räuberische Erpressung auf/gegen Geldinstitute, Postfilialen und -agenturen',
 'Raub, räuberische Erpressung auf/gegen sonstige Zahlstellen und Geschäfte',
 'Handtaschenraub',
 'Sonstige Raubüberfälle auf Straßen, Wegen oder Plätzen',
 'Raubüberfälle in Wohnungen',
 'Gefährliche und schwere Körperverletzung, Verstümmelung weiblicher Genitalien §§ 224, 226, 226a, 231 StGB',
 'Vorsätzliche einfache Körperverletzung § 223 StGB',
 'Diebstahl ohne erschwerende Umstände §§ 242, 247, 248a-c StGB und zwar:',
 'Einfacher Ladendiebstahl',
 'Diebstahl unter erschwerenden Umständen §§ 243-244a StGB und zwar:',
 'Wohnungseinbruchdiebstahl §§ 244 Abs. 1 Nr. 3 und Abs. 4, 244a StGB',
 'Tageswohnungseinbruchdiebstahl §§ 244 Abs. 1 Nr. 3 und A

In [7]:
def cln_strng(strn):
    return strn.replace(' ','').replace(',','').replace('.','').replace('/','').replace('-','').replace(':','').replace('(','').replace(')','').replace('ä','ae').replace('ö','oe').replace('(','').replace('ü','ue').replace('§','')

In [75]:
#Beobachtungen:
# Gewaltkriminalität: gute Aufklärungsquote in Bayern -> wenige Fälle
# Computerkriminalität: Wilhelmshaven und Spree

# Betrug: Schwerin und Erfurt
# Betrug: Schwerin und Erfurt

# Unterschlagung: Brandenburg

# Rauschgiftdelikte: Jena, Mannheim, Würzburg


In [11]:

#field_att='Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst)'


for field_att in allStraf:
    data_fil = data[data.Straftat==field_att].copy()
    krs_tj_sim["RS_int"]=krs_tj_sim["RS"].astype(int)
    join_df= krs_tj_sim.merge(data_fil, left_on='RS_int',right_on='Gemeindeschluessel', how='left')



    #relativ
    field_rel='erfasste Faelle'


    #join_df['field_EWZ']=join_df['Tatverdaechtige insgesamt'].astype(int)/join_df['EWZ']
    join_df['field_EWZ']=join_df[field_rel].str.replace(',','').astype(int)/join_df['EWZ']
    join_df['field_EWZ_rel']=join_df['field_EWZ'].apply(lambda n: '{:,.3%}'.format(n))
    
    join_df['EWZ_format']=join_df['EWZ'].apply(lambda x: '{:,}'.format(x))


    df=join_df

    field="field_EWZ"

    colorscale = branca.colormap.LinearColormap(['green','yellow','red'], index=None, vmin=(df[field]).min(), vmax=(df[field]).max(), caption='')


    def style_function(feature):
        col=feature['properties'][field]
        return {
            'fillOpacity': 0.5,
            'weight': 0,
            'fillColor': 'grey' if col is None else colorscale(col)
        }

    m = folium.Map(location=[51, 12], tiles="Stamen Toner", zoom_start=6)

    folium.GeoJson(
        df,
        name='elternzeit',
        tooltip=folium.GeoJsonTooltip(fields=['GEN','EWZ_format',field_rel,'field_EWZ_rel','Aufklaerungsquote'],
                                      aliases=['Kreis','Einwohner',field_rel,'Anteil','Aufklaerungsquote']),
        style_function=style_function
    ).add_to(m)

    colorscale.caption = field_att
    m.add_child(colorscale)

    
    clean_name= cln_strng(field_att)
    m.save('html/pks2018krs_{}.html'.format(clean_name))


In [12]:
############### CREATE README ###################

file = open("README.md","w",) 
 

file.write("# pks_viz \n")

file.write("PKS - Polizeiliche Kriminalstatistik\n")
file.write("Darstellung auf Kreiebene \n")

file.write("\n")


site_url = "https://smetzler.github.io/pks_viz/index.html"
file.write("## Webseite: \n{} \n".format(site_url))


file.write("\n")

bka_url= "https://www.bka.de/DE/AktuelleInformationen/StatistikenLagebilder/PolizeilicheKriminalstatistik/PKS2018/BKATabellen/bkaTabellenLaenderKreiseStaedteFaelle.html"
geo_url = "https://gdz.bkg.bund.de/index.php/default/digitale-geodaten/verwaltungsgebiete/verwaltungsgebiete-1-250-000-mit-einwohnerzahlen-ebenen-stand-31-12-vg250-ew-ebenen-31-12.html"
file.write("## Quellen: \n")
file.write("Polizeiliche Kriminalstatistik: \n{} \n".format(bka_url))
file.write("Kreis-Shapes und Einwohnerzahlen:\n{} \n".format(geo_url))


file.close() 

In [13]:
############### CREATE INDEX ###################


import dominate
from dominate.tags import *

doc = dominate.document(title='PKS 2018')

with doc.head:
    #<meta charset="UTF-8">  
    #meta('charset="utf-8"')
    link(rel='stylesheet', href="https://www.w3schools.com/w3css/4/w3.css")
    link(rel='stylesheet', href="https://fonts.googleapis.com/css?family=Lato")
    link(rel='stylesheet', href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css")
    link(rel='stylesheet', type="text/css", href="style.css")

    #script(type='text/javascript', src='script.js')

    
with doc:
    with div(id='header'):#.add(ol()):
        h1('PKS 2018: Darstellung auf Kreisebene')


    with div().add(ol()):
        attr(cls='body')
        for field_att in allStraf:
    
            clean_name= cln_strng(field_att)

            pre= "https://smetzler.github.io/pks_viz/html/pks2018krs_"
            link =pre+clean_name

            #file.write("[" + field_att+"]"+link+"\n")
            
            li(a(field_att.title(), href='{}.html'.format(link)))

# save the file again
with open("index.html", "w", encoding='utf-8') as outf:
    outf.write(str(doc))
#print(doc)